In [19]:
import os
import sys
import synapseclient
import pandas as pd

sys.path.append("/Users/apratap/dev/appys/lib/")
import utils
import synapseutils

In [20]:
syn = synapseclient.login()

Welcome, Abhishek Pratap!



In [ ]:
#download all the zip folders uploaded by Serkan and create individual files
for synid in ('syn5562403', 'syn5020954', 'syn5562404', 'syn5562405'):
    zip_folder = syn.get(synid)
    !unzip $zip_folder.path -d test

In [ ]:
rsem_files = utils.get_FilesList("./test/",pattern=".results")

In [ ]:
_ = [syn.store(synapseclient.File(f, parentId = 'syn5020951'), executed='syn5269281') for f in rsem_files]

In [3]:
mdata = syn.query('select id,name from entity where parentId == "syn5020951"')
mdata = synapseutils.query2df(mdata)
mdata = mdata[mdata.name.str.endswith('genes.results')]

### generate TPM values for RSEM mapped data

In [28]:
expression_data = []
for rsem_file in mdata.id:
    rsem_file = syn.get(rsem_file)
    sample_name = os.path.basename(rsem_file.path).replace('.rsem.genes.results','')
    d = pd.read_csv(rsem_file.path, sep="\t")
    d = d.ix[:, ['gene_id','TPM']]
    d.columns = ['gene_id', sample_name]
    d.gene_id = d.gene_id.str.upper()
    d.set_index(['gene_id'],inplace=True)
    expression_data.append(d)
    
rsem_TPM = pd.concat(expression_data, axis=1)
rsem_TPM.to_csv('MGH_RNASeq_samples_TPM_counts.csv', sep="\t")
syn.store(synapseclient.File("MGH_RNASeq_samples_TPM_counts.csv", parentId ="syn5273816" ),
         used=mdata.id.tolist())


##################################################
 Uploading file to Synapse storage 
##################################################
Uploaded Chunks [####################]100.00%     17.2MB/17.2MB MGH_RNASeq_samples_TPM_counts.csv Done...
Upload completed in 4 seconds.


File(contentType=None, parentId=u'syn5273816', synapseStore=True, fileSize=None, versionNumber=6, dataFileHandleId=u'7248010', accessControlList=u'/repo/v1/entity/syn5273852/acl', etag=u'2a47dbf2-3f54-49bd-a78d-6edad9832798', externalURL=None, cacheDir='', creationDate=u'1447357968385', entityType=u'org.sagebionetworks.repo.model.FileEntity', path='MGH_RNASeq_samples_TPM_counts.csv', uri=u'/entity/syn5273852/annotations', uri=u'/repo/v1/entity/syn5273852', concreteType='org.sagebionetworks.repo.model.FileEntity', versions=u'/repo/v1/entity/syn5273852/version', createdOn=u'2015-11-12T19:52:48.385Z', md5=None, files=['MGH_RNASeq_samples_TPM_counts.csv'], annotations=u'/repo/v1/entity/syn5273852/annotations', versionUrl=u'/repo/v1/entity/syn5273852/version/6', name=u'MGH_RNASeq_samples_TPM_counts.csv', versionLabel=u'6', id=u'syn5273852', modifiedOn=u'2015-12-30T03:49:55.614Z', createdBy=u'2166046', modifiedBy=u'2166046', etag=u'7cf22ed0-4721-4392-b583-dbc282190c6b')

### generate expected_count values for RSEM mapped data

In [30]:
expected_count_data = []
for rsem_file in mdata.id:
    rsem_file = syn.get(rsem_file)
    sample_name = os.path.basename(rsem_file.path).replace('.rsem.genes.results','')
    d = pd.read_csv(rsem_file.path, sep="\t")
    d = d.ix[:, ['gene_id','expected_count']]
    d.columns = ['gene_id', sample_name]
    d.gene_id = d.gene_id.str.upper()
    d.set_index(['gene_id'],inplace=True)
    expected_count_data.append(d)
    
rsem_expected_counts = pd.concat(expected_count_data, axis=1)
rsem_expected_counts.to_csv('MGH_RNASeq_samples_RSEM_expected_counts.csv', sep="\t")
syn.store(synapseclient.File("MGH_RNASeq_samples_RSEM_expected_counts.csv", parentId ="syn5273816" ),
         used=mdata.id.tolist())


##################################################
 Uploading file to Synapse storage 
##################################################
Uploaded Chunks [####################]100.00%     18.1MB/18.1MB MGH_RNASeq_samples_RSEM_expected_counts.csv Done...
Upload completed in 4 seconds.


File(contentType=None, createdOn=u'2015-12-29T00:41:10.191Z', etag=u'ea412a81-01c2-49d2-97db-237b1d8aae7a', parentId=u'syn5273816', synapseStore=True, fileSize=None, versionLabel=u'3', versionNumber=3, externalURL=None, cacheDir='', uri=u'/repo/v1/entity/syn5568205', dataFileHandleId=u'7248018', entityType=u'org.sagebionetworks.repo.model.FileEntity', files=['MGH_RNASeq_samples_RSEM_expected_counts.csv'], creationDate=u'1451349670191', accessControlList=u'/repo/v1/entity/syn5568205/acl', etag=u'3a9312d0-cfe7-42e3-8789-11807954e432', concreteType='org.sagebionetworks.repo.model.FileEntity', id=u'syn5568205', path='MGH_RNASeq_samples_RSEM_expected_counts.csv', name=u'MGH_RNASeq_samples_RSEM_expected_counts.csv', modifiedBy=u'2166046', md5=None, versionUrl=u'/repo/v1/entity/syn5568205/version/3', versions=u'/repo/v1/entity/syn5568205/version', uri=u'/entity/syn5568205/annotations', annotations=u'/repo/v1/entity/syn5568205/annotations', modifiedOn=u'2015-12-30T03:52:15.359Z', createdBy=u'2

In [32]:
! rm MGH_RNASeq_samples_TPM_counts.csv MGH_RNASeq_samples_RSEM_expected_counts.csv

In [33]:
! rm -rf test/

### Move bam and bigwig files to folders

In [35]:
mdata = syn.query('select id,name from entity where parentId == "syn4516568"')
mdata = synapseutils.query2df(mdata)
bams = mdata[mdata.name.str.endswith('bam')]
bigwigs = mdata[mdata.name.str.endswith('bw')]

for bam in bams.id.tolist():
    x = syn.get(bam, downloadFile=False)
    x.parentId = 'syn5567576'
    _ = syn.store(x)
    
for bigwig in bigwigs.id.tolist():
    x = syn.get(bigwig, downloadFile=False)
    x.parentId = 'syn5567583'
    _ = syn.store(x)

### Create the meta data tables for MGH RNA Seq samples

In [40]:
batch1 = syn.get('syn5569839')
batch1 = pd.read_excel(batch1.path)

batch1.rename(columns={'Sample_ID':'SampleID', 'NF2 status':'NF2_status', 'Synodos_ID':'sampleAlias',
                      'Cell type':'RNA_source'}, inplace=True)
batch1['Treatment'] = 'untreated'
batch1['batch'] = 1
batch1['sample_description'] = 'NA'
batch1['rx_timepoint'] = None
batch1.sampleAlias = batch1.sampleAlias.str.replace('syn', 'Syn')
batch1['cellLine'] = batch1.sampleAlias
batch1.sampleAlias = batch1.sampleAlias.map(str) + '_' + batch1.Treatment.map(str) 

In [41]:
batch2 = syn.get('syn5562408')
batch2 = pd.read_excel(batch2.path, skiprows=2)
batch2['batch'] = 2
batch3 = syn.get('syn5562407')
batch3 = pd.read_excel(batch3.path)
batch3['batch'] = 3

batch3.rename(columns={'Sample ID':'SampleID'}, inplace=True)
batch2.rename(columns={'Bamfile-name':'SampleID'}, inplace=True)
batch3.rename(columns={'RNA source':'RNA_source'}, inplace=True)


temp_df = batch2['NF2 status'].str.split('\s', expand=True)
batch2['NF2_status'] = temp_df.ix[:,1]
batch2['RNA_source'] = temp_df.ix[:,2]
batch3.rename(columns={'NF2 ':'NF2_status'}, inplace=True)

#rx timepoint
batch2.rename(columns={'Rx timepoint':'rx_timepoint'}, inplace=True)
temp_df = batch3.Treatment.str.split(',', expand=True)
batch3.Treatment = temp_df.ix[:,0]
batch3['rx_timepoint'] = temp_df.ix[:,1]

#
batch2.rename(columns={'Drug ': 'Treatment'}, inplace=True)
batch2.Treatment = batch2.Treatment.str.replace('100nM ','')

#
batch3.rename(columns={'Sample description': 'sample_description'}, inplace=True)
batch2['sample_description'] = 'NA'

#fix drug name
batch2.loc[batch2.Treatment.isin(['GSK2126459', 'GSK2126460']),'Treatment'] = 'GSK2126458'
batch3.loc[batch3.Treatment.isin(['GSK2126459', 'GSK2126460']),'Treatment'] = 'GSK2126458'

#alias
batch3['sampleAlias'] = batch3.SampleID.map(lambda x: str(x).strip()) + '_' + batch3.Treatment.map((lambda x: str(x).strip())) 
batch2['sampleAlias'] = batch2['Cell Line-replicate #'].map(lambda x: str(x).strip()) + '_' + batch2.Treatment.map(lambda x: str(x).strip()) 

#cellLine
batch2['cellLine'] = batch2.sampleAlias.str.split('-',expand=True).ix[:,0]
batch3['cellLine'] = batch3.SampleID.str.split('-',expand=True).ix[:,0]
batch2.cellLine = batch2.cellLine.str.replace(' ','')
batch3.cellLine = batch3.cellLine.str.replace(' ','')

batch2.RNA_source = batch2.RNA_source.str.replace('(','').str.replace(')','')

batch2.drop(['Row', 'Column', 'NF2 status', '# RNA samples', 'Cell Line-replicate #', 'Alias'], inplace=True, axis=1)
batch3.drop(['Row/Col', 'replicate ID'], inplace=True, axis=1)

In [42]:
mdata = pd.concat([batch1,batch2,batch3])

mdata = mdata[~mdata.SampleID.isnull()]
mdata.SampleID = mdata.SampleID.str.replace(' ','')

In [44]:
#create table on synapse
schema = synapseclient.Schema(name="MGH RNASeq Samples metaData", columns=synapseclient.as_table_columns(mdata),
                             parent = 'syn2347420')
mdata_table = syn.store(synapseclient.Table(schema, mdata))

Uploaded Chunks [####################]100.00%     10.6kB/10.6kB /tmp/tmpYVW6ie Done...
Upload completed in 3 seconds.
 [####################]100.00%     1/1  Done...
    

### May 5, 2016
### Create Summary data files for diff calls generated by MGH

In [4]:
diff_calls1 = pd.read_csv(syn.get('syn6041022').path, sep="\t")
diff_calls2 = pd.read_csv(syn.get('syn6041027').path, sep="\t")
diffExp_calls = pd.concat([diff_calls1, diff_calls2])

In [5]:
diffExp_calls.to_csv('Synodos_RNASeq_diffExp_genes_MGH_generated.tsv', sep="\t", index=False )

In [6]:
r = syn.store(synapseclient.File('Synodos_RNASeq_diffExp_genes_MGH_generated.tsv', parentId = 'syn5273816'),
         used = ['syn6041022', 'syn6041027'], executed = 'syn6041029')


##################################################
 Uploading file to Synapse storage 
##################################################
Uploading [####################]100.00%     99.0MB/99.0MB Synodos_RNASeq_diffExp_genes_MGH_generated.tsv Done...
    

#### MGH generated raw counts

In [80]:
raw_counts_synid = ['syn6041023', 'syn6041025', 'syn6041026']
counts = [ pd.read_csv(syn.get(x).path, sep=",") for x in raw_counts_synid]
counts = [x.drop(['start','chr', 'end'], axis=1) for x in counts]
#counts = [x.set_index(['gene']) for x in counts]

In [89]:
final_counts = reduce(lambda left, right: pd.merge(left, right, on='gene'), counts)
final_counts.to_csv("Synodos_RNASeq_data_rawCounts_MGH_generated.tsv", sep="\t", index=False)

In [87]:
syn.store(synapseclient.File("Synodos_RNASeq_data_rawCounts_MGH_generated.tsv", parentId= 'syn6042902'),
         used=raw_counts_synid,
         executed="")